In [1]:
import pandas as pd
import re,glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import seaborn as sns

In [47]:
def rename(path_name):
    name = path_name[38:]
    name = str(re.sub('(rain2007-2020/|water-level2007-2020/)','',name))
    name = re.sub('.csv','',name)
    return name

def df_process(file):
    df = pd.read_csv(file)
    df['station'] = rename(file)
    return df

def open_concat_csv(folder_path):
    all_filenames = [i for i in glob.glob(os.path.join(folder_path, '*.csv'))]
    df = pd.concat([df_process(f) for f in tqdm(all_filenames)])
    return df

path_rain = "../hii-telemetering-batch-data-master/rain2007-2020/"


In [48]:
rain_df = open_concat_csv(path_rain)
# replace -999 value to NaN
rain_df['rain'] = rain_df["rain"].replace({-999:np.nan})

print(rain_df.shape)
rain_df

100%|██████████| 929/929 [00:02<00:00, 361.53it/s]
(4601312, 3)


,date,rain,station
0,2007-01-01,NaN,SWR028
1,2007-01-02,NaN,SWR028
2,2007-01-03,NaN,SWR028
3,2007-01-04,NaN,SWR028
4,2007-01-05,NaN,SWR028
5,2007-01-06,NaN,SWR028
6,2007-01-07,NaN,SWR028
7,2007-01-08,NaN,SWR028
8,2007-01-09,NaN,SWR028
9,2007-01-10,NaN,SWR028


In [18]:
rain_df_st = pd.DataFrame()
for name,group in rain_df.groupby('station'):
    if rain_df_st.empty:
        rain_df_st = group.set_index("date")[["rain"]].rename(columns={'rain':name})
    else:
        rain_df_st = rain_df_st.join(group.set_index("date")[["rain"]].rename(columns={'rain':name}))

rain_df_st.head()

,ACRU,ANLI,BAKI,BARI,BBHN,BBON,BBUA,BBWN,BCAP,BCNG,...,YOM003,YOM004,YOM005,YOM006,YOM007,YOM008,YOM009,YOM010,YOM011,YOM012
date,,,,,,,,,,,,,,,,,,,,,
2007-01-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-02,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-03,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-04,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-05,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# WATER

In [25]:
path_water = "../hii-telemetering-batch-data-master/water-level2007-2020/"
water_df = open_concat_csv(path_water)

print(water_df.shape)
print(water_df.columns)

100%|██████████| 316/316 [01:04<00:00,  4.88it/s]
(225604800, 4)
Index(['date', 'time', 'wl', 'station'], dtype='object')


In [26]:
water_df['wl'] = water_df['wl'].replace({-999:np.nan})
water_df.head()

,date,time,wl,station
0,2007-01-01,00:00:00,NaN,TBW018
1,2007-01-01,00:10:00,NaN,TBW018
2,2007-01-01,00:20:00,NaN,TBW018
3,2007-01-01,00:30:00,NaN,TBW018
4,2007-01-01,00:40:00,NaN,TBW018


In [56]:
all_filenames = [i for i in glob.glob(os.path.join(path_water, '*.csv'))]
print(rename(i) for i in all_filenames)


<generator object <genexpr> at 0x7f44ae6608d0>
